# RNN - Sentimen Analysis

## Requirements Library

In [ ]:
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install keras
!pip install tensorflow
!pip install seaborn


In [ ]:
# alurnya kaya gini:
# 1. bikin sequence dari teks yang udah dikasi pos
# 2. bikin pad sequence
# 3. bikin rnn
# 4. selesai wkwk

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Load Data

In [2]:
rnn_model = pd.read_json('../dataset/tweet-cleaned.json', orient='index')
rnn_model.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim,sentence,5_tagged,emotion
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,"[cepet, bgt, si…, 10YearsOf5SOS]","[cepet, bgt, si…, .]","Sentence: ""cepet bgt si… 10YearsOf5SOS"" [− T...","[cepet_PROPN, bgt_PROPN, si…_PROPN, 10YearsOf5...",sad
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,"[10YearsOf5SOS, udah, 10, tahun, ajaa, time, f...","[udah, 10, tahun, ajaa, time, flies, .., eh, t...","Sentence: ""10YearsOf5SOS udah 10 tahun ajaa ti...","[10YearsOf5SOS_ADV, udah_VERB, 10_NUM, tahun_N...",sad
5,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,"[NANGIS, BGT, 2011, ENAK, BANGET, ALLAHUU, NGI...","[NANGIS, BGT, 2011, ENAK, BANGET, ALLAHUU, NGI...","Sentence: ""NANGIS BGT 2011 ENAK BANGET ALLAHUU...","[NANGIS_PROPN, BGT_PROPN, 2011_NUM, ENAK_PROPN...",happy
7,Liriknya itu loooo🥺😭\n#10YearsOf5SOS https://t...,Liriknya itu loooo🥺😭\n 10YearsOf5SOS,Liriknya itu loooo🥺😭\n,Liriknya itu loooo🥺😭\n 10YearsOf5SOS,Liriknya itu loooo🥺😭\n,Liriknya itu loooo\n 10YearsOf5SOS,Liriknya itu loooo\n,"[Liriknya, itu, loooo, 10YearsOf5SOS]","[Liriknya, itu, loooo]","Sentence: ""Liriknya itu loooo 10YearsOf5SOS"" ...","[Liriknya_NOUN, itu_DET, loooo_VERB, 10YearsOf...",sad
8,happy #10YearsOf5sos \n\n5sosfam itu fandom ke...,happy 10YearsOf5sos \n\n5sosfam itu fandom ke...,"happy \n\n5sosfam itu fandom kedua, 5sos yang...",happy 10YearsOf5sos \n\n5sosfam itu fandom ke...,"happy \n\n5sosfam itu fandom kedua, 5sos yang...",happy 10YearsOf5sos \n\n5sosfam itu fandom ke...,"happy \n\n5sosfam itu fandom kedua, 5sos yang...","[happy, 10YearsOf5sos, 5sosfam, itu, fandom, k...","[happy, 5sosfam, itu, fandom, kedua, ,, 5sos, ...","Sentence: ""happy 10YearsOf5sos 5sosfam itu fan...","[happy_NOUN, 10YearsOf5sos_NOUN, 5sosfam_NOUN,...",happy


In [3]:
sentences = rnn_model['sentence'].tolist()
labels = rnn_model['emotion'].tolist()

In [4]:
rnn_model['emotion'].unique()

array(['sad', 'happy', 'angry', 'surprise', 'funny', 'hopeful', 'disgust',
       'neutral', 'fear'], dtype=object)

In [5]:
sad = rnn_model['emotion'][rnn_model.emotion == 'sad' ]
happy = rnn_model['emotion'][rnn_model.emotion == 'happy' ]
angry = rnn_model['emotion'][rnn_model.emotion == 'angry' ]
surprise = rnn_model['emotion'][rnn_model.emotion == 'surprise' ]
funny = rnn_model['emotion'][rnn_model.emotion == 'funny' ]
hopeful = rnn_model['emotion'][rnn_model.emotion == 'hopeful' ]
disgust = rnn_model['emotion'][rnn_model.emotion == 'disgust' ]
neutral = rnn_model['emotion'][rnn_model.emotion == 'neutral' ]
fear = rnn_model['emotion'][rnn_model.emotion == 'fear' ]


print('Total length of the data is:         {}'.format(rnn_model.shape[0]))
print('No. of sad tagged sentences is:  {}'.format(len(sad)))
print('No. of happy tagged sentences is: {}'.format(len(happy)))
print('No. of angry tagged sentences is:  {}'.format(len(angry)))
print('No. of surprise tagged sentences is: {}'.format(len(surprise)))
print('No. of funny tagged sentences is:  {}'.format(len(funny)))
print('No. of hopeful tagged sentences is: {}'.format(len(hopeful)))
print('No. of disgust tagged sentences is:  {}'.format(len(disgust)))
print('No. of neutral tagged sentences is: {}'.format(len(neutral)))
print('No. of fear tagged sentences is:  {}'.format(len(fear)))

Total length of the data is:         457
No. of sad tagged sentences is:  23
No. of happy tagged sentences is: 240
No. of angry tagged sentences is:  28
No. of surprise tagged sentences is: 10
No. of funny tagged sentences is:  29
No. of hopeful tagged sentences is: 29
No. of disgust tagged sentences is:  42
No. of neutral tagged sentences is: 40
No. of fear tagged sentences is:  16


In [6]:
print(sentences[1])
print(labels[1])

Sentence: "10YearsOf5SOS udah 10 tahun ajaa time flies eh tapi aku baru dengerin lagu mereka pas 2014 sih seingetku"   [− Tokens: 18  − Token-Labels: "10YearsOf5SOS <ADV> udah <VERB> 10 <NUM> tahun <NOUN> ajaa <PROPN> time <PROPN> flies <PROPN> eh <PROPN> tapi <CCONJ> aku <PRON> baru <ADJ> dengerin <NOUN> lagu <NOUN> mereka <PRON> pas <ADJ> 2014 <NUM> sih <PART> seingetku <ADV>"]
sad


In [7]:
from collections import Counter
all_words = []
for line in list(rnn_model['sentence']):
    words = line.split()
    for word in words:
      if(len(word)>2):
        all_words.append(word.lower())
    
    
Counter(all_words).most_common(20)

[('<propn>', 2842),
 ('<noun>', 1418),
 ('<verb>', 650),
 ('sentence:', 457),
 ('tokens:', 457),
 ('token-labels:', 457),
 ('<adv>', 412),
 ('<punct>"]', 387),
 ('<pron>', 337),
 ('<adp>', 264),
 ('<adj>', 221),
 ('<part>', 151),
 ('jeno', 146),
 ('dan', 142),
 ('<num>', 132),
 ('ini', 132),
 ('<det>', 115),
 ('sama', 104),
 ('brightwinxaaa2021', 104),
 ('8yearswithjeno', 103)]

In [8]:
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[:training_size]
training_labels = labels[:training_size]
testing_sentences = sentences[training_size:]
testing_labels = labels[training_size:]

# put labels into list to use later:

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

## Tokenizing And Pad

In [9]:
vocab_size = 1000
embedding_dim = 16
max_length = 280
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [11]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [12]:
print("Train Data size:", len(training_sequences))
print("Test Data size", len(testing_sequences))
print("Vocab size:", vocab_size)
print("Max length:", max_length)

print('tp:',len(training_padded))
print('tp:',len(testing_padded))


Train Data size: 365
Test Data size 92
Vocab size: 1000
Max length: 280
tp: 365
tp: 92


## Evaluate Model

In [13]:
def model_Evaluate(model):
    #accuracy of model on training data
    acc_train=model.score(X_train, y_train)
    #accuracy of model on test data
    acc_test=model.score(X_test, y_test)
    
    print('Accuracy of model on training data : {}'.format(acc_train*100))
    print('Accuracy of model on testing data : {} \n'.format(acc_test*100))

    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    categories  = ['sad', 'happy', 'angry', 'surprise', 'funny', 'hopeful', 'disgust', 'neutral', 'fear']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    # sns.heatmap(cf_matrix, annot = labels, cmap = 'Reds',fmt = '',
    #             xticklabels = categories, yticklabels = categories)

    # plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    # plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    # plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [14]:
#logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

### Logistic Regression

In [15]:
lg = LogisticRegression()
history=lg.fit(X_train, y_train)
model_Evaluate(lg)

NameError: name 'X_train' is not defined

### Linear SVM

In [100]:
#linear svc
from sklearn.svm import LinearSVC

In [101]:
svm = LinearSVC()
svm.fit(X_train, y_train)
model_Evaluate(svm)

Accuracy of model on training data : 41.64383561643836
Accuracy of model on testing data : 18.478260869565215 

              precision    recall  f1-score   support

       angry       0.00      0.00      0.00         6
     disgust       0.00      0.00      0.00         5
        fear       0.20      0.50      0.29         4
       funny       0.11      0.12      0.12         8
       happy       0.50      0.25      0.33        52
     hopeful       0.00      0.00      0.00         6
     neutral       0.07      0.17      0.10         6
         sad       0.00      0.00      0.00         3
    surprise       0.00      0.00      0.00         2

    accuracy                           0.18        92
   macro avg       0.10      0.12      0.09        92
weighted avg       0.31      0.18      0.22        92



c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Random Forest

In [104]:
#random forest
from sklearn.ensemble import RandomForestClassifier

In [105]:
rf = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', max_depth=50)
rf.fit(X_train, y_train)
model_Evaluate(rf)

Accuracy of model on training data : 99.45205479452055
Accuracy of model on testing data : 53.2608695652174 

              precision    recall  f1-score   support

       angry       0.00      0.00      0.00         6
     disgust       0.00      0.00      0.00         5
        fear       0.00      0.00      0.00         4
       funny       0.00      0.00      0.00         8
       happy       0.62      0.90      0.73        52
     hopeful       1.00      0.17      0.29         6
     neutral       0.25      0.17      0.20         6
         sad       0.00      0.00      0.00         3
    surprise       0.00      0.00      0.00         2

    accuracy                           0.53        92
   macro avg       0.21      0.14      0.14        92
weighted avg       0.43      0.53      0.45        92



c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### Bernoulli NB

In [110]:
# bernoulli Naive Bayes
from sklearn.naive_bayes import BernoulliNB


In [111]:
nb = BernoulliNB()
nb.fit(X_train, y_train)
model_Evaluate(nb)

Accuracy of model on training data : 13.150684931506849
Accuracy of model on testing data : 8.695652173913043 

              precision    recall  f1-score   support

       angry       0.00      0.00      0.00         6
     disgust       0.00      0.00      0.00         5
        fear       0.00      0.00      0.00         4
       funny       0.12      0.25      0.17         8
       happy       0.42      0.10      0.16        52
     hopeful       0.00      0.00      0.00         6
     neutral       0.06      0.17      0.09         6
         sad       0.00      0.00      0.00         3
    surprise       0.00      0.00      0.00         2

    accuracy                           0.09        92
   macro avg       0.07      0.06      0.05        92
weighted avg       0.25      0.09      0.11        92



c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Aryo\Documents\spadadikti-ta\ta_nlp\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

## Build Model

`What is RNN?`

Recurrent neural networks (RNN) are the state of the art algorithm for sequential data and are used by Apple's Siri and and Google's voice search. It is the first algorithm that remembers its input, due to an internal memory, which makes it perfectly suited for machine learning problems that involve sequential data

`Embedding Layer`

Embedding layer is one of the available layers in Keras. This is mainly used in Natural Language Processing related applications such as language modeling, but it can also be used with other tasks that involve neural networks. While dealing with NLP problems, we can use pre-trained word embeddings such as GloVe. Alternatively we can also train our own embeddings using Keras embedding layer.

`LSTM layer`

Long Short Term Memory networks, usually called “LSTMs” , were introduced by Hochreiter and Schmiduber. These have widely been used for speech recognition, language modeling, sentiment analysis and text prediction. Before going deep into LSTM, we should first understand the need of LSTM which can be explained by the drawback of practical use of Recurrent Neural Network (RNN). So, lets start with RNN.

### RNN Model 1

In [19]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(embedding_dim,
                         return_sequences=True)
))
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.01),
              metrics=['accuracy'])

In [21]:
callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)
num_epochs=10
history = model.fit(training_padded, training_labels_final, epochs=50,validation_data=(testing_padded, testing_labels),callbacks=[checkpoint2])

NameError: name 'checkpoint2' is not defined

### RNN Model 2

In [22]:
rnn_model['sentence']

0       Sentence: "cepet bgt si… 10YearsOf5SOS"   [− T...
2       Sentence: "10YearsOf5SOS udah 10 tahun ajaa ti...
5       Sentence: "NANGIS BGT 2011 ENAK BANGET ALLAHUU...
7       Sentence: "Liriknya itu loooo 10YearsOf5SOS"  ...
8       Sentence: "happy 10YearsOf5sos 5sosfam itu fan...
                              ...                        
3744    Sentence: "maaf pakbu kameramen sblmnya saya m...
3752    Sentence: "JUNGKOOK LOVE U LO NUTUPIN ABS PAS ...
3771    Sentence: "yerimwiyese IYAA PDHL MSI DIPANGGUN...
3772    Sentence: "BST x FAKE LOVE emg terbaik"   [− T...
3836    Sentence: "BadmintonTalk Semifinal menanti sem...
Name: sentence, Length: 457, dtype: object

In [23]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(rnn_model.sentence)
sequences = tokenizer.texts_to_sequences(rnn_model.sentence)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    1   50    5]
 [   0    0    0 ...   16  885   10]
 [   0    0    0 ...    1   50    5]
 ...
 [   0    0    0 ...    1   52    1]
 [   0    0    0 ...    2  710   13]
 [   0    0    0 ...    2 2387    5]]


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(tweets, rnn_model.emotion.values, test_size=0.2, random_state=101)

In [81]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print()
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (365, 200)
y_train (365,)

X_test (92, 200)
y_test (92,)


In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

SAVE_PERIOD = 10
BATCH_SIZE = 20
STEPS_PER_EPOCH = y_train.size / BATCH_SIZE

model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
checkpoint2 = ModelCheckpoint(
    filepath="rnn_model.hdf5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
history = model2.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint2])

In [63]:
rnn_model.dtypes

raw_text                    object
1st_clean_twitter           object
1st_clean_twitter_gensim    object
2nd_punctuation             object
2nd_punctuation_gensim      object
3rd_emoji                   object
3rd_emoji_gensim            object
4th_tokenized               object
4th_tokenized_gensim        object
sentence                    object
5_tagged                    object
emotion                     object
dtype: object

### RNN Model 3

In [64]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

In [65]:
EMBEDDING_DIM = 300
LR = 1e-3
BATCH_SIZE = 1024
EPOCHS = 10

In [66]:
TRAIN_SIZE = 0.8
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH = 30


In [68]:
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

NameError: name 'embeddings_index' is not defined

In [67]:
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                          EMBEDDING_DIM,
                                          weights=[embedding_matrix],
                                          input_length=MAX_SEQUENCE_LENGTH,
                                          trainable=False)

NameError: name 'embedding_matrix' is not defined

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedding_sequences)
x = Conv1D(64, 5, activation='relu')(x)
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)

## Testing